Start by getting more data.

Include both snotel and weather station data.

In [2]:
import pandas as pd

crystal_summit_seasonal = pd.read_csv('data/seasonal_weather_crystal_summit.csv')
crystal_summit_seasonal.head()

,Date PDT,Temp F,RH %,Min mph,Spd mph,Gust mph,Dir deg
0,Apr 24 6:00,23,98,7,18,31,W
1,Apr 24 5:00,23,98,6,19,30,W
2,Apr 24 4:00,23,98,7,16,29,W
3,Apr 24 3:00,23,98,4,12,21,W
4,Apr 24 2:00,25,97,0,2,9,NW


In [3]:
crystal_summit_seasonal.tail()

,Date PDT,Temp F,RH %,Min mph,Spd mph,Gust mph,Dir deg
5642,Sep 1 4:00,57,46,10,17,23,W
5643,Sep 1 3:00,57,43,9,16,23,W
5644,Sep 1 2:00,59,41,11,19,27,W
5645,Sep 1 1:00,59,37,8,17,23,W
5646,Sep 1 0:00,59,36,6,15,23,W


In [4]:
morse_lake_seasonal = pd.read_csv('data/seasonal_snotel_morse_lake.csv')
morse_lake_seasonal.head()

,Date PDT,Temp F,PcpAc in,∆Pcp in,SWE in,∆SWE in,SnoHt in,∆SnoHt in
0,Apr 24 6:00,30.0,NaN,NaN,57.0,-0.5,126.0,1.0
1,Apr 24 5:00,30.0,NaN,NaN,57.1,-0.2,126.0,1.0
2,Apr 24 4:00,31.0,NaN,NaN,57.2,0.0,123.0,-2.0
3,Apr 24 3:00,31.0,NaN,NaN,57.2,0.2,122.0,-3.0
4,Apr 24 2:00,31.0,NaN,NaN,57.2,0.3,124.0,-2.0


In [5]:
morse_lake_seasonal.tail()

,Date PDT,Temp F,PcpAc in,∆Pcp in,SWE in,∆SWE in,SnoHt in,∆SnoHt in
5593,Sep 1 5:00,53.0,86.3,0.0,NaN,NaN,0.0,0.0
5594,Sep 1 4:00,57.0,86.3,0.0,NaN,NaN,0.0,0.0
5595,Sep 1 3:00,62.0,86.3,0.0,NaN,NaN,0.0,0.0
5596,Sep 1 2:00,58.0,86.3,0.0,NaN,NaN,0.0,0.0
5597,Sep 1 1:00,58.0,86.3,0.0,0.0,0.0,0.0,0.0


todo: what happens if I drop columns I don't think are useful? How does that affect the prediciton accuracy? How does that align with my knowledge of snow science?

In [7]:
crystal_summit_seasonal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5647 entries, 0 to 5646
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date PDT  5647 non-null   object
 1   Temp F    5647 non-null   int64 
 2   RH %      5647 non-null   int64 
 3   Min mph   5647 non-null   int64 
 4   Spd mph   5647 non-null   int64 
 5   Gust mph  5647 non-null   int64 
 6   Dir deg   5647 non-null   object
dtypes: int64(5), object(2)
memory usage: 308.9+ KB


In [41]:
morse_lake_seasonal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5598 entries, 0 to 5597
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date PDT   5598 non-null   object 
 1   Temp F     5597 non-null   float64
 2   PcpAc in   5068 non-null   float64
 3   ∆Pcp in    5047 non-null   float64
 4   SWE in     5290 non-null   float64
 5   ∆SWE  in   5266 non-null   float64
 6   SnoHt in   4800 non-null   float64
 7   ∆SnoHt in  4417 non-null   float64
dtypes: float64(7), object(1)
memory usage: 350.0+ KB


Join the dataframes (should have same date)

In [42]:
seasonal_weather = crystal_summit_seasonal.join(morse_lake_seasonal, lsuffix=" NWAC", rsuffix=" SNOTEL")

seasonal_weather = seasonal_weather.drop('Date PDT SNOTEL', axis=1)
seasonal_weather = seasonal_weather.rename(columns={'Date PDT NWAC': 'Date PDT'})

seasonal_weather.head()

,Date PDT,Temp F NWAC,RH %,Min mph,Spd mph,Gust mph,Dir deg,Temp F SNOTEL,PcpAc in,∆Pcp in,SWE in,∆SWE in,SnoHt in,∆SnoHt in
0,Apr 24 6:00,23,98,7,18,31,W,30.0,NaN,NaN,57.0,-0.5,126.0,1.0
1,Apr 24 5:00,23,98,6,19,30,W,30.0,NaN,NaN,57.1,-0.2,126.0,1.0
2,Apr 24 4:00,23,98,7,16,29,W,31.0,NaN,NaN,57.2,0.0,123.0,-2.0
3,Apr 24 3:00,23,98,4,12,21,W,31.0,NaN,NaN,57.2,0.2,122.0,-3.0
4,Apr 24 2:00,25,97,0,2,9,NW,31.0,NaN,NaN,57.2,0.3,124.0,-2.0


Read the danger ratings

In [43]:
seasonal_danger = pd.read_csv('data/danger_ratings.csv')
# only select target area
target_area = 'West Slopes South'
seasonal_danger = seasonal_danger[(seasonal_danger == target_area).any(axis=1)]
# get rid of days with no rating
no_rating = 'NO RATING'
seasonal_danger = seasonal_danger[seasonal_danger['Danger Rating'] != no_rating]
seasonal_danger.head()

,Date,Area,Danger Rating
15,"Apr 15, 2023",West Slopes South,MODERATE
27,"Apr 14, 2023",West Slopes South,LOW
28,"Apr 13, 2023",West Slopes South,MODERATE
43,"Apr 12, 2023",West Slopes South,MODERATE
54,"Apr 11, 2023",West Slopes South,MODERATE


Connect aggregated weather data to danger ratings

In [44]:
# todo: what happens if we change this?
# todo: how do we weight closer days higher?
# number of previous days to aggregate
days_back = 7

training_data = pd.DataFrame([], columns=seasonal_weather.columns)
training_data['Danger Rating'] = None

for idx, row in seasonal_danger.iterrows():
    date = row['Date'].split(',')[0]
    date_start = row['Date'].split(',')[0] + ' 0:00'
    idx = seasonal_weather[(seasonal_weather == date_start).any(axis=1)].index[0]
    days_past = seasonal_weather.iloc[idx+1:idx-1+24*days_back]
    # aggregate the data
    agg_past = days_past.drop('Date PDT', axis=1).mean(numeric_only=True)
    agg_past['Date PDT'] = date
    agg_past['Danger Rating'] = row['Danger Rating']
    # put it into the dataframe
    training_data.loc[-1] = agg_past
    training_data.index = training_data.index+1

# todo: validate to make sure I did this right
training_data.head()

,Date PDT,Temp F NWAC,RH %,Min mph,Spd mph,Gust mph,Dir deg,Temp F SNOTEL,PcpAc in,∆Pcp in,SWE in,∆SWE in,SnoHt in,∆SnoHt in,Danger Rating
145,Apr 15,24.445783,95.451807,6.090361,14.656627,25.951807,NaN,30.23494,NaN,NaN,53.184337,0.362651,123.373494,-0.849398,MODERATE
144,Apr 14,25.138554,96.0,5.915663,14.704819,26.554217,NaN,30.915663,NaN,NaN,52.821687,0.489157,124.222892,-0.596386,LOW
143,Apr 13,26.704819,96.451807,7.114458,16.933735,30.421687,NaN,31.716867,NaN,NaN,52.33253,0.727273,124.819277,-0.042169,MODERATE
142,Apr 12,27.114458,96.198795,7.240964,17.433735,31.343373,NaN,31.73494,NaN,NaN,51.624848,0.906098,124.861446,-0.277108,MODERATE
141,Apr 11,26.783133,95.813253,7.259036,17.03012,30.126506,NaN,31.012048,NaN,NaN,50.721212,0.998171,125.144578,-0.957831,MODERATE


In [45]:
danger_number = { 'NO RATING': 0, 'LOW': 1, 'MODERATE': 2, 'CONSIDERABLE': 3, 'HIGH': 4, 'EXTREME': 5 }

training_data['Danger Rating'] = training_data['Danger Rating'].map(danger_number)
training_data.head()

,Date PDT,Temp F NWAC,RH %,Min mph,Spd mph,Gust mph,Dir deg,Temp F SNOTEL,PcpAc in,∆Pcp in,SWE in,∆SWE in,SnoHt in,∆SnoHt in,Danger Rating
145,Apr 15,24.445783,95.451807,6.090361,14.656627,25.951807,NaN,30.23494,NaN,NaN,53.184337,0.362651,123.373494,-0.849398,2
144,Apr 14,25.138554,96.0,5.915663,14.704819,26.554217,NaN,30.915663,NaN,NaN,52.821687,0.489157,124.222892,-0.596386,1
143,Apr 13,26.704819,96.451807,7.114458,16.933735,30.421687,NaN,31.716867,NaN,NaN,52.33253,0.727273,124.819277,-0.042169,2
142,Apr 12,27.114458,96.198795,7.240964,17.433735,31.343373,NaN,31.73494,NaN,NaN,51.624848,0.906098,124.861446,-0.277108,2
141,Apr 11,26.783133,95.813253,7.259036,17.03012,30.126506,NaN,31.012048,NaN,NaN,50.721212,0.998171,125.144578,-0.957831,2


Clean up NaN's

Replacing with 0 for now, but todo: is there a better way?

In [48]:
training_data = training_data.fillna(0)

,Temp F NWAC,RH %,Min mph,Spd mph,Gust mph,Dir deg,Temp F SNOTEL,PcpAc in,∆Pcp in,SWE in,∆SWE in,SnoHt in,∆SnoHt in,Danger Rating
145,24.445783,95.451807,6.090361,14.656627,25.951807,0,30.234940,0.000000,0.000000,53.184337,0.362651,123.373494,-0.849398,2
144,25.138554,96.000000,5.915663,14.704819,26.554217,0,30.915663,0.000000,0.000000,52.821687,0.489157,124.222892,-0.596386,1
143,26.704819,96.451807,7.114458,16.933735,30.421687,0,31.716867,0.000000,0.000000,52.332530,0.727273,124.819277,-0.042169,2
142,27.114458,96.198795,7.240964,17.433735,31.343373,0,31.734940,0.000000,0.000000,51.624848,0.906098,124.861446,-0.277108,2
141,26.783133,95.813253,7.259036,17.030120,30.126506,0,31.012048,0.000000,0.000000,50.721212,0.998171,125.144578,-0.957831,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,26.084337,83.445783,9.144578,19.608434,32.927711,0,31.036145,11.475301,0.301205,6.836747,0.349398,19.227273,0.766667,3
3,29.253012,79.355422,8.722892,18.843373,32.162651,0,30.722892,11.156024,0.241566,6.455422,0.249398,18.838462,0.700000,2
2,31.867470,68.512048,7.903614,16.759036,27.873494,0,29.186747,10.896386,0.223494,6.175301,0.227108,18.052632,0.413793,2
1,34.180723,58.000000,7.216867,15.542169,26.150602,0,26.512048,10.655422,0.204217,5.918072,0.196988,17.522059,-0.008197,3


In [47]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

# date isn't a number, and isn't relevant (yet)
try:
    training_data = training_data.drop('Date PDT', axis=1)
except:
    pass

drop = 'Danger Rating'
x = training_data.drop(drop, axis=1)
y = training_data[drop]

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)

rf_regressor = RandomForestRegressor(random_state=0)

rf_regressor.fit(x_train, y_train)

y_pred_train = rf_regressor.predict(x_train)
r2_score(y_train, y_pred_train)


ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values